# SQLNoir — Mysteries Walkthrough (Queries + Inline Commentary)

This notebook contains **7 mysteries**, each with **5 steps** in the pattern  
**SCOPE → NARROW → EVIDENCE → SHORT LIST → CULPRIT**.

- Each step has a **Markdown cell with investigator notes**, followed by a **code cell** that runs the SQL.
- Edit the connection cell below if your password/port differ.
- Results will render as DataFrames directly in the notebook.


In [1]:
# 🔌 Connection (edit if needed)
from sqlalchemy import create_engine, URL, text
import pandas as pd

def make_engine():
    url = URL.create(
        "mssql+pyodbc",
        username="sa",
        password="Str0ng\!Passw0rd",   # <-- change if different
        host="localhost",             # use 'host.docker.internal' if running this INSIDE a container
        port=1433,
        database="SQLNoir",
        query={
            "driver": "ODBC Driver 18 for SQL Server",
            "Encrypt": "yes",
            "TrustServerCertificate": "yes",
        },
    )
    return create_engine(url, pool_pre_ping=True)

engine = make_engine()

def run(sql: str) -> pd.DataFrame:
    """Execute SQL and return a DataFrame."""
    with engine.connect() as conn:
        return pd.read_sql(sql, conn)

# quick smoke test (will print 1 if connection is good)
try:
    with engine.connect() as conn:
        print(conn.execute(text("SELECT 1")).scalar())
except Exception as e:
    print("Connection error:", e)


<>:9: SyntaxWarning: invalid escape sequence '\!'
<>:9: SyntaxWarning: invalid escape sequence '\!'
/var/folders/qz/x8zgt_xj3pv9w0zwf1r5898m0000gn/T/ipykernel_43114/2470525323.py:9: SyntaxWarning: invalid escape sequence '\!'
  password="Str0ng\!Passw0rd",   # <-- change if different


1


## 🕵️ Mystery A — “Blue Note Heist”
**Clues:** trench coat + left-cheek scar; corroborate via interview.

### 1️⃣ Scope — Crime Scene Reference
**Description:** Reads the crime-scene record to justify the filters.  
**How it works:** Anchors the investigation to the primary evidence; confirms trench coat + left-cheek.  
**Why it’s useful:** Ensures later filters are grounded in factual clues.


In [40]:
a1_scope = run(r"""SELECT id, [date], [type], location, description
FROM SQLNoir.crime_scene
WHERE id = 76;""")
a1_scope

,id,date,type,location,description
0,76,19851120,theft,Blue Note Lounge,A briefcase containing sensitive documents van...


### 2️⃣ Narrow — Attire Filter
**Description:** Candidate pool of everyone wearing a trench coat.  
**How it works:** Equality filter on `attire`.  
**Why it’s useful:** High-recall starting set that's easy to narrate.


In [41]:
a2_attire = run(r"""SELECT id, name, attire, scar
FROM SQLNoir.suspects
WHERE attire = 'trench coat';""")
a2_attire

,id,name,attire,scar
0,3,Frankie Lombardi,trench coat,left cheek
1,183,Vincent Malone,trench coat,left cheek
2,237,Christopher Black,trench coat,right cheek


### 3️⃣ Evidence — Add Scar Detail
**Description:** Keep only trench-coat suspects with a left-cheek scar.  
**How it works:** Conjunctive filter (`attire` AND `scar`).  
**Why it’s useful:** Precision filtering → shortlist.


In [42]:
a3_conjunct = run(r"""SELECT id, name, attire, scar
FROM SQLNoir.suspects
WHERE attire = 'trench coat'
  AND scar = 'left cheek';""")
a3_conjunct

,id,name,attire,scar
0,3,Frankie Lombardi,trench coat,left cheek
1,183,Vincent Malone,trench coat,left cheek


### 4️⃣ Short List — Set Logic Cross-check
**Description:** Recreate result using `INTERSECT`.  
**How it works:** Intersection of two simple selects (trench ∩ left-cheek).  
**Why it’s useful:** Teaches set theory equivalence to AND.


In [43]:
a4_intersect = run(r"""SELECT id, name, attire, scar FROM SQLNoir.suspects WHERE attire = 'trench coat'
INTERSECT
SELECT id, name, attire, scar FROM SQLNoir.suspects WHERE scar = 'left cheek';""")
a4_intersect

,id,name,attire,scar
0,3,Frankie Lombardi,trench coat,left cheek
1,183,Vincent Malone,trench coat,left cheek


### 5️⃣ Culprit — Interviews Join & Rank
**Description:** Attach interviews; rank confessional phrasing higher.  
**How it works:** **LEFT JOIN** suspects→interviews on `suspect_id`; order transcripts containing “steal”/“did it.”  
**Why it’s useful:** Combines quantitative filters with qualitative text; `LEFT JOIN` keeps silent suspects.


In [44]:
a5_culprit = run(r"""SELECT TOP (1) s.id, s.name, s.attire, s.scar, i.transcript
FROM SQLNoir.suspects s
LEFT JOIN SQLNoir.interviews i ON i.suspect_id = s.id
WHERE s.attire = 'trench coat' AND s.scar = 'left cheek'
ORDER BY
  CASE WHEN i.transcript LIKE '%steal%' OR i.transcript LIKE '%did it%' THEN 0 ELSE 1 END,
  s.name;""")
a5_culprit

,id,name,attire,scar,transcript
0,183,Vincent Malone,trench coat,left cheek,"I wasn't going to steal it, but I did."


## 🕵️ Mystery 2 — “Luxury Hotel Grab”
**Clues:** suit + right-cheek scar; demonstrate EXCEPT (A − B).

### 1️⃣ Scope — Crime Scene Reference
**Description:** Validate suit + right-cheek in case text.  
**How it works:** Read the specific crime-scene row.  
**Why it’s useful:** Grounds later filters.


In [45]:
m2_1_scope = run(r"""SELECT id, [date], [type], location, description
FROM SQLNoir.crime_scene
WHERE id = 14;""")
m2_1_scope

,id,date,type,location,description
0,14,20060907,theft,Luxury Hotel,A briefcase was stolen from a guest. The thief...


### 2️⃣ Base Set A — All Suits
**Description:** Collect the suit cohort.  
**How it works:** Equality on `attire`.  
**Why it’s useful:** Defines minuend for subtraction.


In [46]:
m2_2_suits = run(r"""WITH Suits AS (
  SELECT id, name, attire, scar
  FROM SQLNoir.suspects
  WHERE attire = 'suit'
)
SELECT * FROM Suits;""")
m2_2_suits

,id,name,attire,scar
0,2,Tony DeMarco,suit,none
1,9,Joshua Smith,suit,none
2,16,Laura Wright,suit,none
3,21,Victor Carter,suit,none
4,27,Penny Turner,suit,none
5,37,Ella King,suit,none
6,49,Benjamin White,suit,none
7,57,Maddison Lewis,suit,none
8,63,Benjamin Miller,suit,right cheek
9,71,Lily White,suit,none


### 3️⃣ Build Set B — Contradictions
**Description:** Suits whose scar is **not** right cheek.  
**How it works:** `scar <> 'right cheek'`.  
**Why it’s useful:** Explicit to-be-removed subset.


In [47]:
m2_3_notright = run(r"""WITH NotRightCheek AS (
  SELECT id, name, attire, scar
  FROM SQLNoir.suspects
  WHERE attire = 'suit' AND scar <> 'right cheek'
)
SELECT * FROM NotRightCheek;""")
m2_3_notright

,id,name,attire,scar
0,2,Tony DeMarco,suit,none
1,9,Joshua Smith,suit,none
2,16,Laura Wright,suit,none
3,21,Victor Carter,suit,none
4,27,Penny Turner,suit,none
5,37,Ella King,suit,none
6,49,Benjamin White,suit,none
7,57,Maddison Lewis,suit,none
8,71,Lily White,suit,none
9,81,Matthew Jackson,suit,none


### 4️⃣ Short List — Set Subtraction (A − B)
**Description:** Keep suits minus contradictions.  
**How it works:** `EXCEPT` models the must-be-right-cheek rule.  
**Why it’s useful:** Clean, teachable set subtraction.


In [48]:
m2_4_shortlist = run(r"""WITH Suits AS (
  SELECT id, name, attire, scar FROM SQLNoir.suspects WHERE attire = 'suit'
),
NotRightCheek AS (
  SELECT id, name, attire, scar FROM SQLNoir.suspects WHERE attire = 'suit' AND scar <> 'right cheek'
)
SELECT * FROM Suits
EXCEPT
SELECT * FROM NotRightCheek;""")
m2_4_shortlist

,id,name,attire,scar
0,63,Benjamin Miller,suit,right cheek


### 5️⃣ Culprit — Enrich with Interviews
**Description:** Join shortlist to suspects for names, then add transcripts.  
**How it works:** `JOIN` shortlist→suspects (metadata), then `LEFT JOIN` interviews on `suspect_id`.  
**Why it’s useful:** Connects who the rows represent and what they said.


In [49]:
m2_5_joined = run(r"""WITH Suits AS (
  SELECT id, name, attire, scar FROM SQLNoir.suspects WHERE attire = 'suit'
),
NotRightCheek AS (
  SELECT id, name, attire, scar FROM SQLNoir.suspects WHERE attire = 'suit' AND scar <> 'right cheek'
),
FinalList AS (
  SELECT * FROM Suits
  EXCEPT
  SELECT * FROM NotRightCheek
)
SELECT s.name, s.attire, s.scar, i.transcript
FROM   FinalList f
JOIN   SQLNoir.suspects  s ON s.id = f.id
LEFT   JOIN SQLNoir.interviews i ON i.suspect_id = f.id;""")
m2_5_joined

,name,attire,scar,transcript
0,Benjamin Miller,suit,right cheek,None


## 🕵️ Mystery 3 — “Old Town Square Run”
**Clues:** right-hand scar + coat/jacket attire.

### 1️⃣ Scope — Scar Anchor
**Description:** Start with right-hand scars.  
**How it works:** Equality on `scar`.  
**Why it’s useful:** Unique physical marker.


In [50]:
m3_1_scope = run(r"""SELECT id, name, attire, scar
FROM SQLNoir.suspects
WHERE scar = 'right hand';""")
m3_1_scope

,id,name,attire,scar
0,42,Mila Roberts,sweater,right hand
1,77,David Davis,sweater,right hand
2,110,Oliver Carter,sweater,right hand
3,263,Benjamin Walker,jacket,right hand


### 2️⃣ Narrow — Coat/Jacket Family
**Description:** Keep trench/jacket/blazer/tuxedo only.  
**How it works:** `IN` list on `attire`.  
**Why it’s useful:** Encodes “some kind of coat.”


In [51]:
m3_2_narrow = run(r"""SELECT id, name, attire, scar
FROM SQLNoir.suspects
WHERE scar = 'right hand'
  AND attire IN ('trench coat','jacket','blazer','tuxedo');""")
m3_2_narrow

,id,name,attire,scar
0,263,Benjamin Walker,jacket,right hand


### 3️⃣ Evidence — Interviews Join
**Description:** Attach transcripts to read for context (alibi, place).  
**How it works:** **LEFT JOIN** suspects→interviews on `suspect_id`.  
**Why it’s useful:** Merges behavioral clues with physical filters.


In [52]:
m3_3_interviews = run(r"""SELECT s.id, s.name, s.attire, s.scar, i.transcript
FROM SQLNoir.suspects s
LEFT JOIN SQLNoir.interviews i ON i.suspect_id = s.id
WHERE s.scar = 'right hand'
  AND s.attire IN ('trench coat','jacket','blazer','tuxedo');""")
m3_3_interviews

,id,name,attire,scar,transcript
0,263,Benjamin Walker,jacket,right hand,None


### 4️⃣ Short List — Readable Roll-call
**Description:** IDs/names of finalists for narration.  
**How it works:** Project identifiers only.  
**Why it’s useful:** Clean on-screen list.


In [53]:
m3_4_shortlist = run(r"""SELECT id, name
FROM SQLNoir.suspects
WHERE scar = 'right hand'
  AND attire IN ('trench coat','jacket','blazer','tuxedo');""")
m3_4_shortlist

,id,name
0,263,Benjamin Walker


### 5️⃣ Culprit — Deterministic Pick
**Description:** Stable selection with alphabetic tie-breaker.  
**How it works:** `ORDER BY name` then `TOP (1)`.  
**Why it’s useful:** Reproducible grading outcome.


In [54]:
m3_5_culprit = run(r"""SELECT TOP (1) id, name, attire, scar
FROM SQLNoir.suspects
WHERE scar = 'right hand'
  AND attire IN ('trench coat','jacket','blazer','tuxedo')
ORDER BY name;""")
m3_5_culprit

,id,name,attire,scar
0,263,Benjamin Walker,jacket,right hand



   ## MYSTERY 4 — "Metro Mask Getaway"


🕵️ **Mystery Summary:**  
A masked figure dashed through **Central Station**, leaving behind chaos at the turnstiles.  
Witnesses recall trembling hands and a **dark jacket**, while the interview tapes capture raw **nervous energy**.  
This mystery combines **scene evidence** (the mask), **physical appearance** (jacket), and **emotional cues** (panic and sweat) to reveal the culprit.

💡 **Query 1 — SCOPE (Scene Validation):**  
We start by confirming that the crime scene description actually mentions a *mask* at *Central Station*.  
This anchors our case in reality — the details tell us where and what to focus on.  
Verifying this ensures every later filter connects to an authentic narrative clue.



In [55]:
m4_1_all = run(r"""SELECT id, [date], location, description
FROM SQLNoir.crime_scene
WHERE location LIKE '%Station%' AND description LIKE '%mask%';""")
m4_1_all

,id,date,location,description
0,102,20160701,Central Station,A masked figure sprinted through the Central S...


💡 **Query 2 — NARROW (Attire Filter):**  
Next, we pull every suspect known to wear a **jacket**.  
This forms our initial candidate pool — matching the witness’s physical observation at the scene.  
We don’t yet consider their emotions or stories, only appearance.


In [56]:
m4_2_scar = run(r"""SELECT id, name, attire, scar
FROM SQLNoir.suspects
WHERE attire = 'jacket';
""")
m4_2_scar

,id,name,attire,scar
0,4,Carlos Garcia,jacket,right arm
1,13,Sam Johnson,jacket,none
2,22,Mason Clark,jacket,left ear
3,28,Benjamin Lopez,jacket,none
4,38,Lucas Scott,jacket,left cheek
5,46,Lily Jackson,jacket,none
6,52,Sebastian Smith,jacket,left hand
7,61,Sophia Harris,jacket,left thigh
8,66,Jack Mitchell,jacket,right wrist
9,76,Emma Hall,jacket,none


💡 **Query 3 — EVIDENCE (Emotional Cross-Check):**  
Now we bring in the **interview transcripts**.  
Here, we’re listening for linguistic signs of distress — words like *“nervous,” “panic,” “sweat.”*  
These phrases humanize the data, giving insight into who’s emotionally connected to the chaos of that night.



In [57]:
m4_3_coats = run(r"""SELECT s.id, s.name, s.attire, s.scar, i.transcript
FROM SQLNoir.suspects s
LEFT JOIN SQLNoir.interviews i ON i.suspect_id = s.id
WHERE s.attire = 'jacket'
  AND (i.transcript LIKE '%nervous%' OR i.transcript LIKE '%panic%' OR i.transcript LIKE '%sweat%');
""")
m4_3_coats

,id,name,attire,scar,transcript
0,405,Liam Torres,jacket,left cheek,"I was nervous, okay? The mask made it hard to ..."


💡 **Query 4 — SHORT LIST (Behavioral Ranking):**  
We rank each suspect based on how strongly their language expresses anxiety.  
This step lets investigators quantify demeanor — turning tone and vocabulary into measurable suspicion.  
The nervous-score highlights who might have cracked under pressure.


In [58]:
m4_4_interviews = run(r"""SELECT s.id, s.name,
       CASE WHEN i.transcript LIKE '%nervous%' OR i.transcript LIKE '%panic%' OR i.transcript LIKE '%sweat%' THEN 1 ELSE 0 END AS nervous_score
FROM SQLNoir.suspects s
LEFT JOIN SQLNoir.interviews i ON i.suspect_id = s.id
WHERE s.attire = 'jacket'
ORDER BY nervous_score DESC, s.name;
""")
m4_4_interviews

,id,name,nervous_score
0,405,Liam Torres,1
1,146,Benjamin Green,0
2,232,Benjamin Green,0
3,28,Benjamin Lopez,0
4,400,Benjamin Walker,0
5,263,Benjamin Walker,0
6,4,Carlos Garcia,0
7,109,Charlotte Harris,0
8,91,Charlotte King,0
9,188,Chloe Brown,0


💡 **Query 5 — CULPRIT (Final Identification):**  
Finally, we identify the **most anxious jacket-wearer** from the interview pool.  
Their transcript aligns perfectly with the witness account, confirming **Liam Torres** as the nervous masked runner.  
The emotional breakdown and physical evidence converge — case closed.


In [59]:
m4_5_culprit = run(r"""SELECT TOP (1) s.id, s.name, s.attire, s.scar, i.transcript
FROM SQLNoir.suspects s
LEFT JOIN SQLNoir.interviews i ON i.suspect_id = s.id
WHERE s.attire = 'jacket'
  AND (i.transcript LIKE '%nervous%' OR i.transcript LIKE '%panic%' OR i.transcript LIKE '%sweat%')
ORDER BY s.name;
""")
m4_5_culprit

,id,name,attire,scar,transcript
0,405,Liam Torres,jacket,left cheek,"I was nervous, okay? The mask made it hard to ..."



   ## MYSTERY 5 — "The Ballroom Masquerade"


🕵️ **Mystery Summary:**  
At the **Grand Hotel Ballroom**, a glittering masquerade gala turned chaotic when a diamond vanished mid-dance.  
Music stopped, masks fell, and someone in a **tuxedo** stormed off, shouting in anger.  
Our job: trace the elegance, the fury, and the lie hidden behind polished shoes and silver cufflinks.

💡 **Query 1 — SCOPE (Scene Confirmation):**  
We begin by locating the crime scene that mentions both *“mask”* and *“music.”*  
These two words place us squarely in the ballroom gala — an event defined by its disguise and performance.  
Validating this scene helps narrow our investigation to the masquerade setting.

In [60]:
m5_1_suits = run(r"""SELECT id, [date], location, description
FROM SQLNoir.crime_scene
WHERE location LIKE '%Ballroom%' AND description LIKE '%mask%' AND description LIKE '%music%';
""")
m5_1_suits

,id,date,location,description
0,103,20161015,Grand Hotel Ballroom,A stolen diamond disappeared during the masque...


💡 **Query 2 — NARROW (Attire Selection):**  
With the gala context confirmed, we filter suspects who wore **tuxedos**.  
Formal wear instantly connects them to the event’s guest list.  
This step turns a room full of possibilities into a small circle of high-society suspects.



In [61]:
m5_2_visible = run(r"""SELECT id, name, attire, scar
FROM SQLNoir.suspects
WHERE attire = 'tuxedo';
""")
m5_2_visible

,id,name,attire,scar
0,7,Richard Brown,tuxedo,none
1,19,Tommy Walker,tuxedo,none
2,30,Matthew Collins,tuxedo,none
3,41,William Johnson,tuxedo,none
4,50,Dylan Walker,tuxedo,none
5,73,Chloe Anderson,tuxedo,none
6,93,Sebastian Lewis,tuxedo,none
7,111,Emily Taylor,tuxedo,none
8,129,Liam Martin,tuxedo,none
9,141,Olivia Turner,tuxedo,none


💡 **Query 3 — EVIDENCE (Emotional Filter):**  
Here we listen to the words beneath the surface — the **interview transcripts**.  
Expressions like *“I swear,” “please listen,” “they think I yelled”* betray emotional cracks in composure.  
Olivia’s repeated insistence of innocence sounds more rehearsed than truthful, revealing her inner tension.


In [62]:
m5_3_interviews = run(r"""SELECT s.id, s.name, s.attire, s.scar, i.transcript
FROM SQLNoir.suspects s
LEFT JOIN SQLNoir.interviews i ON i.suspect_id = s.id
WHERE s.attire = 'tuxedo'
  AND (i.transcript LIKE '%angry%' OR i.transcript LIKE '%swear%' OR i.transcript LIKE '%yell%');
""")
m5_3_interviews

,id,name,attire,scar,transcript
0,161,Olivia King,tuxedo,none,"I wasn't near the crime scene, I swear."
1,406,Victor Price,tuxedo,right cheek,I swear I didn’t yell at anyone! I was angry b...


💡 **Query 4 — SHORT LIST (Intensity Ranking):**  
Here, we score every tuxedo suspect based on emotional intensity.  
Those whose words show anger or denial rise to the top, forming our shortlist.  
The emotional frequency helps separate the truly pressured from the calm liars.


In [63]:
m5_4_shortlist = run(r"""SELECT s.id, s.name,
       CASE WHEN i.transcript LIKE '%angry%' OR i.transcript LIKE '%swear%' OR i.transcript LIKE '%yell%' THEN 1 ELSE 0 END AS emotion_score
FROM SQLNoir.suspects s
LEFT JOIN SQLNoir.interviews i ON i.suspect_id = s.id
WHERE s.attire = 'tuxedo'
ORDER BY emotion_score DESC, s.name;
""")
m5_4_shortlist

,id,name,emotion_score
0,161,Olivia King,1
1,406,Victor Price,1
2,73,Chloe Anderson,0
3,50,Dylan Walker,0
4,111,Emily Taylor,0
5,214,Ethan Davis,0
6,129,Liam Martin,0
7,383,Lucas Green,0
8,201,Mason Walker,0
9,30,Matthew Collins,0


💡 **Query 5 — CULPRIT (The Defensive Guest):**  
Ultimately, **Olivia King** emerges as the definitive match — her transcript blends grace with barely restrained anxiety.  
Her tone, equal parts poise and protest, mirrors the scene’s dissonance: music silenced, elegance shattered.  
The ballroom lights may have dimmed, but her words illuminated guilt.


In [64]:
m5_5_culprit = run(r"""SELECT TOP (1) s.id, s.name, s.attire, s.scar, i.transcript
FROM SQLNoir.suspects s
LEFT JOIN SQLNoir.interviews i ON i.suspect_id = s.id
WHERE s.attire = 'tuxedo'
  AND (i.transcript LIKE '%angry%' OR i.transcript LIKE '%swear%' OR i.transcript LIKE '%yell%')
ORDER BY s.name;
""")
m5_5_culprit

,id,name,attire,scar,transcript
0,161,Olivia King,tuxedo,none,"I wasn't near the crime scene, I swear."



   ## MYSTERY 6 — "The Rain-Soaked Gallery"


🕵️ **Mystery Summary:**  
On a stormy night at the **City Gallery**, broken glass glittered beside a forgotten **umbrella**.  
A man in a **blazer** lingered near the shattered frame, speaking softly about “art” and “light.”  
The mystery contrasts a calm exterior with the tension of a wet, chaotic night.

💡 **Query 1 — SCOPE (Scene Verification):**  
We first confirm that the gallery’s description includes *“umbrella”* or *“raincoat.”*  
This anchors our evidence to a rainy, emotionally subdued setting.  
Every story must begin with a clear picture of place and atmosphere.


In [65]:
m6_1_suits = run(r"""SELECT id, [date], location, description
FROM SQLNoir.crime_scene
WHERE location LIKE '%Gallery%' AND (description LIKE '%umbrella%' OR description LIKE '%raincoat%');
""")
m6_1_suits

,id,date,location,description
0,104,20170311,City Gallery,"During a rainy night, an umbrella was found be..."


💡 **Query 2 — NARROW (Attire Selection):**  
Then we isolate suspects wearing **blazers**, aligning with the witness’s sighting near the gallery.  
This attire choice already suggests refinement — an ideal cover for someone who blends into an art crowd.  
We move from dozens of suspects to a single stylish handful.


In [66]:
m6_2_rankscar = run(r"""SELECT id, name, attire, scar
FROM SQLNoir.suspects
WHERE attire = 'blazer';
""")
m6_2_rankscar

,id,name,attire,scar
0,15,Daniel King,blazer,left thigh
1,26,Andrew Moore,blazer,right cheek
2,35,Charlotte White,blazer,none
3,43,Jackson Mitchell,blazer,none
4,48,Sophie Harris,blazer,none
5,56,Charlotte Robinson,blazer,none
6,62,Grace Scott,blazer,none
7,74,Ava Moore,blazer,none
8,80,Mason Turner,blazer,right knee
9,86,Jack Robinson,blazer,none


💡 **Query 3 — EVIDENCE (Emotional Filter):**  
We pull in interviews to read between the lines.  
Terms like *“painting,” “art,” “quiet”* reflect calm composure — emotional fingerprints of someone at ease.  
This linguistic serenity is suspicious amid the chaos of a break-in.



In [67]:
m6_3_interviews = run(r"""SELECT s.id, s.name, s.attire, s.scar, i.transcript
FROM SQLNoir.suspects s
LEFT JOIN SQLNoir.interviews i ON i.suspect_id = s.id
WHERE s.attire = 'blazer'
  AND (i.transcript LIKE '%painting%' OR i.transcript LIKE '%art%' OR i.transcript LIKE '%quiet%');
""")
m6_3_interviews

,id,name,attire,scar,transcript
0,80,Mason Turner,blazer,right knee,I wasn't part of any of this.
1,407,Elias Ward,blazer,none,The rain was calming. I was just admiring the ...


💡 **Query 4 — SHORT LIST (Intensity Ranking):**  
We score each blazer wearer for artistic or tranquil tone.  
Those whose words resonate with calm reflection rise to the top.  
This is psychological profiling through text — the stillness that betrays involvement.


In [68]:
m6_4_scored = run(r"""SELECT s.id, s.name,
       CASE WHEN i.transcript LIKE '%painting%' OR i.transcript LIKE '%art%' OR i.transcript LIKE '%quiet%' THEN 1 ELSE 0 END AS calm_score
FROM SQLNoir.suspects s
LEFT JOIN SQLNoir.interviews i ON i.suspect_id = s.id
WHERE s.attire = 'blazer'
ORDER BY calm_score DESC, s.name;
""")
m6_4_scored

,id,name,calm_score
0,407,Elias Ward,1
1,80,Mason Turner,1
2,26,Andrew Moore,0
3,74,Ava Moore,0
4,389,Benjamin Brown,0
5,136,Benjamin Harris,0
6,314,Benjamin Taylor,0
7,126,Benjamin White,0
8,273,Benjamin White,0
9,56,Charlotte Robinson,0


💡 **Query 5 — CULPRIT (The Unmasked Dancer):**  
Ultimately, **Elias Ward** fits the puzzle perfectly.  
His gentle, poetic answers mirror the art he supposedly admired — yet also the cold precision of a thief.  
The rain, the frame, and the quiet voice complete the portrait of guilt.


In [69]:
m6_5_culprit = run(r"""SELECT TOP (1) s.id, s.name, s.attire, s.scar, i.transcript
FROM SQLNoir.suspects s
LEFT JOIN SQLNoir.interviews i ON i.suspect_id = s.id
WHERE s.attire = 'blazer'
  AND (i.transcript LIKE '%painting%' OR i.transcript LIKE '%art%' OR i.transcript LIKE '%quiet%')
ORDER BY s.name;
""")
m6_5_culprit

,id,name,attire,scar,transcript
0,407,Elias Ward,blazer,none,The rain was calming. I was just admiring the ...


## 🕵️ Mystery 7 — “Harbor Warehouse Job”
**Clues:** blazer + right-cheek scar; prefer transcripts mentioning ‘Harbor’/‘Dock’. 

### 1️⃣ Scope — Blazer Cohort
**Description:** Start with all blazer wearers.  
**How it works:** Filter `attire='blazer'`.  
**Why it’s useful:** Frames the attire context near the docks.


In [70]:
m7_1_blazers = run(r"""SELECT id, name, attire, scar
FROM SQLNoir.suspects
WHERE attire = 'blazer';""")
m7_1_blazers

,id,name,attire,scar
0,15,Daniel King,blazer,left thigh
1,26,Andrew Moore,blazer,right cheek
2,35,Charlotte White,blazer,none
3,43,Jackson Mitchell,blazer,none
4,48,Sophie Harris,blazer,none
5,56,Charlotte Robinson,blazer,none
6,62,Grace Scott,blazer,none
7,74,Ava Moore,blazer,none
8,80,Mason Turner,blazer,right knee
9,86,Jack Robinson,blazer,none


### 2️⃣ Narrow — Right-Cheek Scar
**Description:** Keep only blazer wearers with a right-cheek scar.  
**How it works:** Conjunctive condition on `scar`.  
**Why it’s useful:** Strong distinguishing physical clue.


In [71]:
m7_2_rightcheek = run(r"""SELECT id, name, attire, scar
FROM SQLNoir.suspects
WHERE attire = 'blazer' AND scar = 'right cheek';""")
m7_2_rightcheek

,id,name,attire,scar
0,26,Andrew Moore,blazer,right cheek
1,401,Marcus “Dockside” Hale,blazer,right cheek
2,403,Marcus 'Dockside' Hale,blazer,right cheek
3,404,Marcus Hale,blazer,right cheek


### 3️⃣ Evidence — Interviews Join
**Description:** Attach transcripts for scene corroboration.  
**How it works:** **LEFT JOIN** suspects→interviews on `suspect_id`.  
**Why it’s useful:** Enables keyword hunting for locations/behavior.


In [72]:
m7_3_interviews = run(r"""SELECT s.id, s.name, s.scar, i.transcript
FROM SQLNoir.suspects s
LEFT JOIN SQLNoir.interviews i ON i.suspect_id = s.id
WHERE s.attire = 'blazer' AND s.scar = 'right cheek';""")
m7_3_interviews

,id,name,scar,transcript
0,26,Andrew Moore,right cheek,"I wasn't even in the area, sorry."
1,401,Marcus “Dockside” Hale,right cheek,None
2,403,Marcus 'Dockside' Hale,right cheek,"Look, everyone hangs near Dock 7 after shift. ..."
3,404,Marcus Hale,right cheek,"Look, everyone hangs near Dock 7 after shift. ..."


### 4️⃣ Short List — Scene Mention Score
**Description:** Score: transcript mentions “Harbor”/“Dock” = 1; sort score desc then name.  
**How it works:** Text match on transcript to compute `mentions_scene`.  
**Why it’s useful:** Prioritizes location-consistent statements.


In [76]:
m7_4_scene = run(r"""
SELECT DISTINCT s.name,
       CASE WHEN i.transcript LIKE '%Harbor%' OR i.transcript LIKE '%Dock%' THEN 1 ELSE 0 END AS mentions_scene
FROM SQLNoir.suspects s
LEFT JOIN SQLNoir.interviews i ON i.suspect_id = s.id
WHERE s.attire = 'blazer' AND s.scar = 'right cheek'
ORDER BY mentions_scene DESC, s.name;
""")
m7_4_scene

,name,mentions_scene
0,Marcus 'Dockside' Hale,1
1,Marcus Hale,1
2,Andrew Moore,0
3,Marcus “Dockside” Hale,0


### 5️⃣ Culprit — Final with Context
**Description:** Highest scene-score candidate (tiebreak by name); display transcript.  
**How it works:** Deterministic ordering by `mentions_scene` then name.  
**Why it’s useful:** Answer carries the most contextually aligned narrative proof.


In [74]:
m7_5_culprit = run(r"""SELECT TOP (1) s.id, s.name, s.attire, s.scar, i.transcript
FROM SQLNoir.suspects s
LEFT JOIN SQLNoir.interviews i ON i.suspect_id = s.id
WHERE s.attire = 'blazer' AND s.scar = 'right cheek'
ORDER BY
  CASE WHEN i.transcript LIKE '%Harbor%' OR i.transcript LIKE '%Dock%' THEN 0 ELSE 1 END,
  s.name;""")
m7_5_culprit

,id,name,attire,scar,transcript
0,403,Marcus 'Dockside' Hale,blazer,right cheek,"Look, everyone hangs near Dock 7 after shift. ..."
